# Comparing the performance of optimizers

In [ ]:
import pennylane as qml
import numpy as np
from qiskit import IBMQ
import itertools
import matplotlib.pyplot as plt
import pickle
import scipy

## Hardware-friendly circuit

In [ ]:
n_wires = 5

In [ ]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='...', group='...', project='...')

In [ ]:
n_shots_list = [10, 100, 1000]
devs = [qml.device('qiskit.ibmq', wires=n_wires, backend='ibmq_valencia', provider=provider, shots=shots) for shots in n_shots_list]
devs.append(qml.device("default.qubit", wires=n_wires))

In [ ]:
devs

In [ ]:
def layers_circ(weights):
    for i in range(n_wires):
        qml.RX(weights[i], wires=i)

    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[2, 1])
    qml.CNOT(wires=[3, 1])
    qml.CNOT(wires=[4, 3])
    return qml.expval(qml.PauliZ(1))

In [ ]:
layers = [qml.QNode(layers_circ, d) for d in devs]

In [ ]:
seed = 2

weights = qml.init.basic_entangler_layers_uniform(n_layers=1, n_wires=5, seed=seed).flatten()
weights

In [ ]:
layers[0](weights)

In [ ]:
grads = [qml.grad(l, argnum=0) for l in layers]

In [ ]:
g_exact = np.round(grads[-1](weights), 7)
g_exact

## Calculating the Hessian

In [ ]:
s = 0.5 * np.pi
denom = 4 * np.sin(s) ** 2
shift = np.eye(len(weights))


def hess_gen_results(func, weights, args=None):
    
    results = {}
    
    if not args:
        args = len(weights)
    
    for c in itertools.combinations(range(args), r=2):
        weights_pp = weights + s * (shift[c[0]] + shift[c[1]])
        weights_pm = weights + s * (shift[c[0]] - shift[c[1]])
        weights_mp = weights - s * (shift[c[0]] - shift[c[1]])
        weights_mm = weights - s * (shift[c[0]] + shift[c[1]])

        f_pp = func(weights_pp)
        f_pm = func(weights_pm)
        f_mp = func(weights_mp)
        f_mm = func(weights_mm)
        results[c] = (f_pp, f_mp, f_pm, f_mm)
    
    f = func(weights)
    
    for i in range(args):
        f_p = func(weights + 0.5 * np.pi * shift[i])
        f_m = func(weights - 0.5 * np.pi * shift[i])
        results[(i, i)] = (f_p, f_m, f)

    return results


def hess_diag_gen_results(func, weights, args=None):
    
    results = {}
    
    if not args:
        args = len(weights)
    
    f = func(weights)
    
    for i in range(args):
        f_p = func(weights + 0.5 * np.pi * shift[i])
        f_m = func(weights - 0.5 * np.pi * shift[i])
        results[(i, i)] = (f_p, f_m, f)

    return results


def grad_gen_results(func, weights, args=None):
    results = {}
    
    if not args:
        args = len(weights)
    
    for i in range(args):
        f_p = func(weights + 0.5 * np.pi * shift[i])
        f_m = func(weights - 0.5 * np.pi * shift[i])
        results[i] = (f_p, f_m)
    
    return results


def get_hess_diag(func, weights, args=None):
    if not args:
        args = len(weights)
        
    hess = np.zeros(args)
    results = hess_diag_gen_results(func, weights, args)
    
    for i in range(args):
        r = results[(i, i)]
        hess[i] = (r[0] + r[1] - 2 * r[2]) / 2
    
    grad = np.zeros(args)
    
    for i in range(args):
        r = results[(i, i)]
        grad[i] = (r[0] - r[1]) / 2
    
    return hess, results, grad


def get_grad(func, weights, args=None):
    
    if not args:
        args = len(weights)
    
    grad = np.zeros(args)
    results = grad_gen_results(func, weights, args)
    
    for i in range(args):
        r = results[i]
        grad[i] = (r[0] - r[1]) / 2
    
    return results, grad
    
    
def get_hess(func, weights, args=None):
    
    if not args:
        args = len(weights)
        
    hess = np.zeros((args, args))
    
    results = hess_gen_results(func, weights, args)
    
    for c in itertools.combinations(range(args), r=2):
        r = results[c]
        hess[c] = (r[0] - r[1] - r[2] + r[3]) / denom
    
    hess = hess + hess.T
    
    for i in range(args):
        r = results[(i, i)]
        hess[i, i] = (r[0] + r[1] - 2 * r[2]) / 2
    
    grad = np.zeros(args)
    
    for i in range(args):
        r = results[(i, i)]
        grad[i] = (r[0] - r[1]) / 2
    
    return hess, results, grad

In [ ]:
hess_ps = get_hess(layers[-1], weights)
print(np.around(hess_ps[0], 3))
print(np.around(hess_ps[2], 3))
print(np.allclose(g_exact, hess_ps[2]))

In [ ]:
hess_ps = get_hess_diag(layers[-1], weights)
print(np.around(hess_ps[0], 3))
print(np.around(hess_ps[2], 3))
print(np.allclose(g_exact, hess_ps[2]))

In [ ]:
grad_ps = get_grad(layers[-1], weights)
print(np.around(grad_ps[1], 3))
print(np.allclose(g_exact, grad_ps[1]))

Test with just first two params

In [ ]:
hess_ps = get_hess(layers[-1], weights, 2)
print(np.around(hess_ps[0], 3))
print(np.around(hess_ps[2], 3))
print(np.allclose(g_exact[:2], hess_ps[2]))

In [ ]:
hess_ps = get_hess_diag(layers[-1], weights, 2)
print(np.around(hess_ps[0], 3))
print(np.around(hess_ps[2], 3))
print(np.allclose(g_exact[:2], hess_ps[2]))

In [ ]:
grad_ps = get_grad(layers[-1], weights, 2)
print(np.around(grad_ps[1], 3))
print(np.allclose(g_exact[:2], grad_ps[1]))

## Visualizing optimization surface

In [ ]:
grid = 200
xs = np.linspace(- 2 * np.pi, 2 * np.pi, grid)
ys = np.linspace(- 2 * np.pi, 2 * np.pi, grid)

xv, yv = np.meshgrid(xs, ys)
zv = np.zeros((grid, grid))

for i in range(grid):
    for j in range(grid):
        w = weights.copy()
        w[0] = xv[i, j]
        w[1] = yv[i, j]
        zv[i, j] = layers[-1](w)

In [ ]:
np.savez("grid.npz", xs=xs, ys=ys, zv=zv)

In [ ]:
g = np.load("grid.npz")
xs = g["xs"]
ys = g["ys"]
zv = g["zv"]

In [ ]:
weights[0] = 0.1
weights[1] = 0.15

In [ ]:
weights

In [ ]:
def gradient_descent(func, weights, reps, lr, i, args=2):
    ws = [weights.copy()]
    res_dict = {}
    gs = []
    costs = [func(weights)]
    
    for r in range(reps):
        res, g = get_grad(func, ws[-1], args)
        res_dict[r] = res
        gs.append(g)
        
        w_updated = ws[-1].copy()
        w_updated[:args] -= lr * g
        
        ws.append(w_updated)
        costs.append(func(w_updated))
        
        if r % 5 == 0:
            print("Calculated for repetition {}".format(r))
    
        with open("gds_results_{}.pickle".format(i), "wb") as f:
            pickle.dump([ws, res, gs, costs], f)
    
    return ws, res_dict, gs, costs

In [ ]:
reps = 50
lr = 0.4
args = 2

for i, l in enumerate(layers):    
    print("Calculating for layer {}".format(i))
    ws, res, gs, costs = gradient_descent(l, weights, reps, lr, i, 2)

In [ ]:
def newton(func, weights, reps, lr, iii, args=2):
    ws = [weights.copy()]
    res_dict = {}
    gs = []
    hs = []
    costs = [func(weights)]
    
    for r in range(reps):
        hess_r, res, g = get_hess(func, ws[-1], args)
        
        res_dict[r] = res
        gs.append(g)
        hs.append(hess_r)
        
        w_updated = ws[-1].copy()
        
        h_inv = np.linalg.inv(hess_r + np.eye(2))
        w_updated[:args] -= lr * h_inv @ g
        
        ws.append(w_updated)
        costs.append(func(w_updated))
        
        if r % 5 == 0:
            print("Calculated for repetition {}".format(r))
    
        with open("new_results_{}.pickle".format(iii), "wb") as f:
            pickle.dump([ws, res, gs, hs, costs], f)
    
    return ws, res_dict, gs, hs, costs

In [ ]:
reps = 50
lr = 0.4
args=2

for i, l in enumerate(layers):   
    print("Calculating for layer {}".format(i))
    ws, res, gs, hs, costs = newton(l, weights, reps, lr, i, 2)

In [ ]:
def newton_diag(func, weights, reps, lr, iii, args=2):
    ws = [weights.copy()]
    res_dict = {}
    gs = []
    hs = []
    costs = [func(weights)]
    
    for r in range(reps):

        hess_r, res, g = get_hess_diag(func, ws[-1], args)
        
        res_dict[r] = res
        gs.append(g)
        hs.append(hess_r)
        
        w_updated = ws[-1].copy()
        
        update = lr * g / (1.0 + hess_r)
        for i in range(len(update)):
            if np.isinf(update[i]):
                update[i] = 0
                
        w_updated[:args] -= update
     
        
        ws.append(w_updated)
        costs.append(func(w_updated))
        
        if r % 5 == 0:
            print("Calculated for repetition {}".format(r))
    
        with open("new_d_results_{}.pickle".format(iii), "wb") as f:
            pickle.dump([ws, res, gs, hs, costs], f)
    
    return ws, res_dict, gs, hs, costs

In [ ]:
reps = 50
lr = 0.4
args=2

for i, l in enumerate(layers):  
    print("Calculating for layer {}".format(i))
    ws, res, gs, hs, costs = newton_diag(l, weights, reps, lr, i, 2)